In [1]:
import configparser
import os
import logging
import time
from datetime import datetime
import pandas as pd
import hmac
import hashlib
import requests
from requests import Session, Request, Response
from urllib.parse import urlencode
from typing import Optional, Dict, Any
from datetime import timedelta
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
"""Implement classes for communication with exchanges.
Description
----------
Implements a class to execute trades on an exchange by communicating via API.
Implements a class to execute trades on an exchange by communicating
via API.
Classes
----------
    ExchangeClient:
        Provides an interface for the trading bot to communicate
        with an exchange via its' API.
Functions
----------
    Doesn't implement any module functions.
Exceptions
----------
    Exports no exceptions.
"""


class ExchangeClient:
    """Provides an interface for the trading bot to communicate with an exchange via its' API."""
    """Provide a Client for the exchange.
    
    Provide an interface for the trading bot to communicate with an exchange
    via its' API.
    ...
    
    Attributes
    ----------
    maker_fees_USD_futures : float
        The maker fees for the exchange in USD.
    taker_fees_USD_futures : float
        The taker fees for the exchange in USD.
    Methods
    -------
    open_session() -> None:
        Opens a session with the exchange server.
    read_config() -> None:
        Initializes the Exchange client.
    get(path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        Executes an HTTPS GET request.
    post(path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        Executes an HTTPS POST request.
    delete(path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        Executes an HTTPS DELETE request.
    place_order(market: str, side: str, size: float, reduce_only: bool = False,
                order_type: str = 'MARKET') -> dict:
        Places a buy or sell order with the exchange.
    """

    def __init__(self, endpoint: str):
        """
        Parameters
        ----------
        endpoint : str
            The endpoint of the exchange.
        """

        self.maker_fees_USD_futures = 0.0002
        self.taker_fees_USD_futures = 0.0004
        self._session = None
        self._config = None
        self._api_key = None
        self._api_secret = None
        self._endpoint = endpoint
        
        
    def open_session(self) -> None:
        """Opens a session with the exchange server."""
        
        self._session = Session()


    def get(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        """Executes an HTTPS GET request."""
        """Executes an HTTPS GET request.
        
        Parameters
        ----------
        path : str
            The path of the request.
        params : Optional[Dict[str, Any]]
            The parameters of the request.
        
        Returns
        -------
        Any
            The response of the request.
        """

        return self._request('GET', path, params=params)


    def post(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        """Executes an HTTPS POST request."""
        """Executes an HTTPS POST request.
        
        Parameters
        ----------
        path : str
            The path of the request.
        params : Optional[Dict[str, Any]]
            The parameters of the request.
        
        Returns
        -------
        Any
            The response of the request.
        """

        return self._signed_request('POST', path, params=params)


    def fetch_current_data(self, market_name: str, timeframe: str) -> pd.DataFrame:

        response = self.get(f'fapi/v1/klines?symbol={market_name}&interval={timeframe}')
        df = pd.DataFrame(response, columns = ['open time', 'open', 'high', 'low', 'close', 
                                                'volume', 'close time', 'quote asset volume', 
                                                'number of trades', 'taker buy base asset volume',
                                                'taker buy quote asset volume', 'unused field'])

        df['open time'] = df['open time'].apply(self._adjust_ts)
        df['open time'] = df['open time'].apply(datetime.fromtimestamp)
        df[['open', 'high', 'low', 'close', 'volume']] \
                = df[['open', 'high', 'low', 'close', 'volume']].astype(float)

        return df
        return self._signed_request('DELETE', path, json=params)

    def _adjust_ts(self, ts: datetime.timestamp) -> datetime.timestamp:
        """Adjusts timestamps from milliseconds to seconds."""

        ts /= 1000
        return ts


    def _process_response(self, response: Response) -> Any:
        """Processes the response the server sends to the clients request."""
        """Processes the response the server sends to the clients request.
        
        Parameters
        ----------
        response : Response
            The response of the request.
            
        Returns
        -------
        Any
            The response of the request.
        """

        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            return data

    def _request(self, method: str, path: str, **kwargs) -> Any:
        """Executes a non-confidential request to the exchange server."""
        """Executes a non-confidential request to the exchange server.
        
        Parameters
        ----------
        method : str
            The method of the request.
        path : str
            The path of the request.
        **kwargs
            The parameters of the request.
            
        Returns
        -------
        Any
            The response of the request.
        """

        request = Request(method, self._endpoint + path, **kwargs)

        try:
            response = self._session.send(request.prepare())
        except requests.exceptions.Timeout:
            logging.warning("Request has timed out")
        except requests.exceptions.ConnectionError:
            logging.warning("Request has faced a connection error")
            
        return self._process_response(response)

In [3]:
def fetch_historical_data(ec: ExchangeClient, start_time: datetime, end_time: datetime):
    response = ec.get(f'fapi/v1/klines?symbol=ETHUSDT&interval=1d'
                      + f'&startTime={int(start_time.timestamp()*1000)}'
                      + f'&endTime={int(end_time.timestamp()*1000)}&limit={1000}')
    quotes = pd.DataFrame(response, columns = ['open time', 'open', 'high', 'low', 'close', 
                                               'volume', 'close time', 'quote_asset_volume', 
                                               'number of trades', 'taker_buy_base_asset_volume',
                                               'taker_buy_quote_asset_volume', 'unused field'])

    quotes[['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume',
            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']] \
        = quotes[['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume',
                'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']].astype(float)

    #while start_time < datetime.now():
    funding = None
    response = ec.get(f'fapi/v1/fundingRate?symbol=ETHUSDT'
                      + f'&startTime={int((start_time-timedelta(days=1)).timestamp()*1000)}'
                      + f'&endTime={int(end_time.timestamp()*1000)}')
    funding = pd.DataFrame(response, columns = ['fundingTime', 'fundingRate'])
    funding['fundingRate'] = funding['fundingRate'].astype(float)
    
    return quotes, funding

In [4]:
ec = ExchangeClient(endpoint = 'https://fapi.binance.com/')
ec.open_session()

delta = timedelta(days=40)
start_time = datetime(2019, 11, 28, 20, 0, 0, 0)
end_time = start_time + delta

quotes = pd.DataFrame()
funding = pd.DataFrame()

while start_time < datetime.now():

    tmp_quotes, tmp_funding = fetch_historical_data(ec, start_time, end_time)

    quotes = pd.concat([quotes, tmp_quotes], ignore_index=True)
    funding = pd.concat([funding, tmp_funding], ignore_index=True)

    start_time = end_time
    end_time = end_time + delta


print(quotes.iloc[:5])
print(funding.iloc[:5])

       open time    open    high     low   close      volume     close time  \
0  1574985600000  150.56  157.40  150.55  154.41  167906.104  1575071999999   
1  1575072000000  154.40  155.15  149.66  151.38  370491.615  1575158399999   
2  1575158400000  151.38  152.50  145.50  150.65  394494.119  1575244799999   
3  1575244800000  150.72  151.30  146.67  148.59  395630.910  1575331199999   
4  1575331200000  148.59  149.80  145.62  147.10  495996.573  1575417599999   

   quote_asset_volume  number of trades  taker_buy_base_asset_volume  \
0        2.582493e+07             31539                    69723.600   
1        5.653920e+07             44923                   135195.907   
2        5.874181e+07             46005                   143318.592   
3        5.882911e+07             42498                   145520.355   
4        7.343436e+07             47707                   226933.611   

   taker_buy_quote_asset_volume unused field  
0                  1.074863e+07            0 

In [5]:
# Convert the 'open time' column in the first data frame to datetime and set it as index
quotes['date'] = pd.to_datetime(quotes['open time']/1000, unit='s').dt.date
quotes.set_index('date', inplace=True)

# Convert the 'fundingTime' column in the second data frame to datetime and set it as index
funding['date'] = pd.to_datetime(funding['fundingTime']/1000, unit='s').dt.date
funding.set_index('date', inplace=True)

# Sort the two data frames by their timestamp columns
quotes.sort_values('open time', inplace=True)
funding.sort_values('fundingTime', inplace=True)

# Merge the two data frames based on the date index
#merged_df = pd.merge(quotes, funding, how='outer', left_index=True, right_index=True)

quotes['open time'] = quotes['open time'].astype('int64')
funding['fundingTime'] = funding['fundingTime'].astype('int64')
# Use merge_asof to match the closest timestamp in df2 to each entry in df1
merged_df = pd.merge_asof(quotes, funding, left_on='open time', right_on='fundingTime')

# Drop the redundant 'date' column
# merged_df.drop(columns=['date'], inplace=True)

# Print the merged data frame
print(merged_df[['open time', 'fundingTime', 'open', 'high', 'low', 'close', 'fundingRate']])


          open time    fundingTime     open     high      low    close  \
0     1574985600000  1574985600000   150.56   157.40   150.55   154.41   
1     1575072000000  1575072000000   154.40   155.15   149.66   151.38   
2     1575158400000  1575158400000   151.38   152.50   145.50   150.65   
3     1575244800000  1575244800000   150.72   151.30   146.67   148.59   
4     1575331200000  1575331200000   148.59   149.80   145.62   147.10   
...             ...            ...      ...      ...      ...      ...   
1283  1685836800000  1685836800000  1890.90  1914.53  1883.19  1888.54   
1284  1685923200000  1685894400000  1888.54  1890.01  1775.00  1810.03   
1285  1686009600000  1686009600000  1810.03  1898.00  1796.00  1883.26   
1286  1686096000000  1686096000000  1883.26  1897.20  1820.00  1831.31   
1287  1686182400000  1686153600000  1831.32  1861.41  1827.42  1856.16   

      fundingRate  
0       -0.000228  
1        0.000001  
2        0.000100  
3        0.000100  
4        0.

In [9]:
df = merged_df[-400:]

df['algo funding'] = 100*365*df['fundingRate']

# Replace 'df' with the name of your dataframe
window_size = 55
df['p'] = 100 * (df['close'] - df['low'].rolling(window_size).min()) / (df['high'].rolling(window_size).max() - df['low'].rolling(window_size).min())


# define a function to count consecutive negative values
def count_negatives(x):
    count = 0
    for i in x:
        if i < 0:
            count += 1
        else:
            count = 0
        yield count
        
# define a function to count consecutive positive values
def count_positives(x):
    count = 0
    for i in x:
        if i >= 0:
            count += 1
        else:
            count = 0
        yield count
        
# create a new column with the count of consecutive negative values
df['bear_count'] = list(count_negatives(df['algo funding']))

df['bull_count'] = list(count_positives(df['algo funding']))

# calculate the 110-day moving average of column A
df['MA110'] = df['close'].rolling(window=110).mean()
df['MA200'] = df['close'].rolling(window=200).mean()
df['bull_value'] = df['low'].rolling(window=50).mean()

df['bear_value'] = 0.7 * df['MA110']

# Set the initial mode as 1
mode = 1

# Set the initial counter as 0
counter_low = 0
counter_high = 0

# Save the sell line and mode values in new columns in the dataframe
df['Mode'] = df['Sell_line'] = pd.Series([0]*len(df))
df['Sell_line'] = df['Sell_line'].astype(float)
df['Mode'] = df['Mode'].astype(float)

for i, row in df.iterrows():
    df.at[i, 'Sell_line'] = row['bull_value'] * mode
    df.at[i, 'Mode'] = mode

    if row['close'] > df['Sell_line'].iloc[i]:
        if counter_low > 0:
            counter_low = 0
            
        if counter_high >= 4:
            if mode < 1.5:
                mode += 0.1
            counter_high = 0
        else:
            counter_high += 1
    else:
        if counter_high > 0:
            counter_high = 0
        
        if counter_low >= 4:
            if mode > 1.1:
                mode -= 0.1
            counter_low = 0
        else:
            counter_low += 1

# define a lambda function to apply to each row
f = lambda row: row['bull_value'] if row['high'] > row['MA200'] else row['bear_value']

# create a new column based on the conditions in col_3 and col_4
df['limit line'] = df.apply(f, axis=1)

weak_buy = lambda row: 1 if (row['algo funding'] < 0
                             and row['bear_count'] < 7 
                             and row['close'] < row['limit line']
                             and (row['close'] > row['MA110'] or row['high'] < row['MA110'])) else 0

df['weak buy'] = df.apply(weak_buy, axis=1)

strong_buy = lambda row: 1 if (row['algo funding'] < 0
                               and row['bear_count'] >= 7 
                               and row['close'] < row['limit line']
                               and (row['close'] > row['MA110'] or row['high'] < row['MA110'])) else 0

df['strong buy'] = df.apply(strong_buy, axis=1)

weak_sell = lambda row: 1 if (row['close'] > row['Sell_line']
                              and row['high'] > row['MA110']
                              and row['bull_count'] < 7
                              and row['p'] > 80) else 0

df['weak sell'] = df.apply(weak_sell, axis=1)

strong_sell = lambda row: 1 if (row['close'] > row['Sell_line']
                                and row['high'] > row['MA110']
                                and row['bull_count'] >= 7
                                and row['p'] > 80) else 0

df['strong sell'] = df.apply(strong_sell, axis=1)

blow_off = lambda row: 1 if (row['high'] > 1.5 * row['bull_value']) else 0

df['blow off top'] = df.apply(blow_off, axis=1)

/tmp/ipykernel_158135/3576550810.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_158135/3576550810.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_158135/3576550810.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykern

IndexError: single positional indexer is out-of-bounds

In [10]:
# create subplots with two y-axes
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(
    x=pd.to_datetime(df['open time'][df['weak buy'] > 0], unit='ms'),
    y=-200+df['close'][df['weak buy'] > 0],
    name='weak buy',
    mode='markers',
    marker=dict(
        symbol='triangle-up',
        size=10,
        color='blue',
        line=dict(width=1, color='black'),
    )
))

fig.add_trace(go.Scatter(
    x=pd.to_datetime(df['open time'][df['strong buy'] > 0], unit='ms'),
    y=-200+df['close'][df['strong buy'] > 0],
    name='strong buy',
    mode='markers',
    marker=dict(
        symbol='triangle-up',
        size=10,
        color='green',
        line=dict(width=1, color='black'),
    )
))

"""
fig.add_trace(go.Scatter(
    x=df['open time'][df['weak sell'] > 0],
    y=1000+df['close'][df['weak sell'] > 0],
    name='weak sell',
    mode='markers',
    marker=dict(
        symbol='triangle-down',
        size=10,
        line=dict(width=1, color='black'),
    )
))
"""

fig.add_trace(go.Scatter(
    x=pd.to_datetime(df['open time'][df['strong sell'] > 0], unit='ms'),
    y=200+df['close'][df['strong sell'] > 0],
    name='strong sell',
    mode='markers',
    marker=dict(
        symbol='triangle-down',
        size=10,
        color='red',
        line=dict(width=1, color='black'),
    )
))

fig.add_trace(go.Scatter(
    x=pd.to_datetime(df['open time'][df['blow off top'] > 0], unit='ms'),
    y=200+df['close'][df['blow off top'] > 0],
    name='blow off top',
    mode='markers',
    marker=dict(
        symbol='triangle-down',
        size=10,
        color='orange',
        line=dict(width=1, color='black'),
    )
))

# add the trace for the close data to the first y-axis
fig.add_trace(go.Candlestick(
    x=pd.to_datetime(df['open time'], unit='ms'),
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close'],
    name='Close')
)

fig.add_trace(go.Scatter(
    x=pd.to_datetime(df['open time'], unit='ms'),
    y=df['limit line'],
    mode='lines',
    name='limit line',
    line=dict(color='green')
))


fig.add_trace(go.Scatter(
    x=pd.to_datetime(df['open time'], unit='ms'),
    y=df['Sell_line'],
    mode='lines',
    name='sell line',
    line=dict(color='red')
))

# set the layout of the figure
fig.update_layout(title='Close and Funding Rate', xaxis_title='Date', yaxis_title='Price')

# show the plot
fig.show()

In [11]:
# Create the first subplot
fig = make_subplots(rows=2,
                    cols=1,
                    subplot_titles=("Plot 1", "Plot 2"),
                    specs=[[{"secondary_y": True}], [{"secondary_y": True}]])

fig.add_trace(go.Scatter(
    x=pd.to_datetime(df['open time'], unit='ms'),
    y=df['bull_count'],
    mode='lines',
    name='bull count',
    line=dict(color='green')
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=pd.to_datetime(df['open time'], unit='ms'),
    y=-df['bear_count'],
    mode='lines',
    name='bear count',
    line=dict(color='red')
), row=2, col=1)


fig.add_trace(go.Scatter(
    x=pd.to_datetime(df['open time'], unit='ms'),
    y=df['algo funding'],
    mode='lines',
    name='algo funding',
    line=dict(color='blue')
), row=2, col=1, secondary_y=True)


fig.add_trace(go.Scatter(
    x=pd.to_datetime(df['open time'], unit='ms'),
    y=df['close'],
    mode='lines',
    name='close',
    line=dict(color='blue')
), row=1, col=1)

# set the layout of the figure
fig.update_layout(title='Close and Funding Rate', xaxis_title='Date', yaxis_title='Price')

# show the plot
fig.show()